# Build Topic Model
The present notebook will use the data collected in section 5.1 to compute and explore a topic model. In essence, a topic model consists of two sets of probability distributions. First, the probability that a *word* belongs to a certain topic (Topic/Term probability). Second, the probability of a *topic* to appear in a document (Document/Topic probability). These probabilities are never 0, that is, each word has some probability (even if very small) to be part of each topic and each topic has some probability (even if very small) to appear in a document.

A standard way to inspect a topic model is to look at the top-ten words of a topic (the ten words with the highest probability in each particular topic). Similarly, we can pull the ten most important documents for each topic (the documents in which this topic has the highest probability).

For a more thorough analysis we may create full probability tables: a topic/term probability table and a document/topic probability table. These tables give a fuller account of the model and will be used for the visualizations (5.3). 

# TODO
Look at https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for ideas about hyperparameters, perplexity, convergence, etc. and https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/ for visualizations.

# INSTALL
gensim needs installation

Also look into saving a model to disk in Gensim.

In [1]:
import numpy as np
import pandas as pd
from gensim import corpora, models, utils
import gensim
import pickle

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [2]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [3]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis. Note that this list of stop words was assembled for the SAAo corpus - another corpus may require a different list, or none at all. In a cell further below the dictionary is built - leaving out words that appear in more than 80 percent of the documents (or whatever the 'no_above' parameter is set too) making the use of a stop word list mostly unnecessary. The only advantage of an explicit list of stop words is that it makes it possible to filter out documents or text fragments that remain with too few words to be meaningful.

The 'stoplist' cell can be omitted entirely or adapted to your purposes.

In [4]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have at least 10 lemmas left and use that as a mask to filter  the list `texts` as well as the dataframe `df`. 

In [5]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [6]:
len(bo), len(df)

(5009, 3033)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.save('output/ldadict')
## CHECK - is this done correctly?

In [8]:
corpus = [dictionary.doc2bow(doc) for doc in texts]
corpora.MmCorpus.serialize('output/ldacorpus', corpus)

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

The visualization (section 5.3) will fail if the number of topics is higher than 25. 

In [9]:
ntopics = int(input("Number of topics: ") or 10)
if ntopics > 25:
    ntopics = 25

Number of topics:  


In [10]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=150)

List the top 10 words and their probabilities in all topics. Note: the topic numbers here are not the ones used in the visualizations in 5.3! (The topics are the same, but not their numbers).

Save the model

In [11]:
saved = 'output/ldasaved'
ldamodel.save(saved)

In [12]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('manû[unit]n', 0.07460955),
   ('ṣarpu[silver]n', 0.05077602),
   ('hurāṣu[gold]n', 0.047348257),
   ('biltu[load]n', 0.04248671),
   ('šiqlu[unit]n', 0.034077335),
   ('ammatu[unit]n', 0.026702428),
   ('eleppu[ship]n', 0.021947471),
   ('šawiru[ring]n', 0.01901914),
   ('pūtu[forehead]n', 0.018668577),
   ('kāru[quay]n', 0.017603515)]),
 (1,
  [('sisû[horse]n', 0.044651676),
   ('karābu[pray]v', 0.039131243),
   ('dullu[trouble]n', 0.030765027),
   ('ṭūbu[goodness]n', 0.027118457),
   ('šīru[flesh]n', 0.014573547),
   ('balāṭu[live]v', 0.013999887),
   ('ilu[god]n', 0.013509653),
   ('našû[lifted]aj', 0.013248313),
   ('šattu[year]n', 0.011965697),
   ('wadû[know]v', 0.011748885)]),
 (2,
  [('rabû[big]aj', 0.05999384),
   ('ilūtu[divinity]n', 0.03386138),
   ('immeru[sheep]n', 0.03187009),
   ('šalmu[intact]aj', 0.026799057),
   ('luʾʾû[sullied]aj', 0.022594161),
   ('pû[mouth]n', 0.018162629),
   ('bīru[divination]n', 0.016008219),
   ('qātu[hand]n', 0.014118909),
   ('kīnu

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [13]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.32245758),
 (1, 0.6159893),
 (2, 0.007693431),
 (3, 0.0076949983),
 (4, 0.007693014),
 (5, 0.007693492),
 (6, 0.0076945233),
 (7, 0.007696353),
 (8, 0.0076935324),
 (9, 0.007693785)]

# Create Document/Topic Probability Table
A Document/Topic probability table is a table (DataFrame), where each row represents a document and each column a topic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, with as index the index of the original DataFrame with the tokenized data.  

In [14]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
P224395,0.004001,0.240667,0.110501,0.004002,0.004001,0.004002,0.004002,0.620818,0.004001,0.004003
P224403,0.322458,0.615989,0.007693,0.007695,0.007693,0.007693,0.007695,0.007696,0.007694,0.007694
P224417,0.206175,0.002041,0.002042,0.075811,0.002041,0.002041,0.002041,0.703724,0.002042,0.002041
P224431,0.002174,0.002174,0.002174,0.263511,0.002174,0.002174,0.002174,0.719095,0.002174,0.002174
P224433,0.004763,0.004763,0.004763,0.183521,0.004763,0.004763,0.069543,0.713596,0.004763,0.004763


We can use the above table to find the ten highest scoring documents per topic with the pandas function 'nlargest'. First add the 'designation' as a separate column to the table.

In [15]:
d_t_df_w_desig = pd.merge(df['designation'], d_t_df, left_index=True, right_index=True)
d_t_df_w_desig

,designation,0,1,2,3,4,5,6,7,8,9
P224395,Arabs Attack a Column of Booty,0.004001,0.240667,0.110501,0.004002,0.004001,0.004002,0.004002,0.620818,0.004001,0.004003
P224403,Straw and Reeds for Dur-Šarruken,0.322458,0.615989,0.007693,0.007695,0.007693,0.007693,0.007695,0.007696,0.007694,0.007694
P224417,Turning in Taxes and Organizing the Province,0.206175,0.002041,0.002042,0.075811,0.002041,0.002041,0.002041,0.703724,0.002042,0.002041
P224431,Raising Food and Fodder from Desert Towns,0.002174,0.002174,0.002174,0.263511,0.002174,0.002174,0.002174,0.719095,0.002174,0.002174
P224433,Urarṭu After the Cimmerian Rout,0.004763,0.004763,0.004763,0.183521,0.004763,0.004763,0.069543,0.713596,0.004763,0.004763
...,...,...,...,...,...,...,...,...,...,...,...
P452805,You Did Well With the Bit-Amukanians,0.007144,0.545832,0.007146,0.007145,0.007147,0.397005,0.007146,0.007145,0.007144,0.007147
P452901,Invading Elam (646-XII-27),0.007695,0.007695,0.007695,0.007696,0.007697,0.007697,0.413542,0.443770,0.088817,0.007696
Q009252,Confiscating Scholarly Tablets in Borsippa,0.036023,0.001451,0.001450,0.001450,0.493684,0.001451,0.321610,0.139980,0.001451,0.001451
X210106,Granting the Kingship of Dilmun (647-VI-13),0.002327,0.002327,0.002327,0.002327,0.002327,0.045755,0.791208,0.115421,0.033654,0.002327


The following code goes through the (numbered) columns of the table which hold the probabilities of each of the topics (the columns) in each of the documents (the rows). The highest ten probabilities are selected, together with a brief descriptipon of the text (designation). 

In [16]:
doctop = []
for i in range(ntopics):
    t = d_t_df_w_desig.nlargest(10, i)[['designation', i]]
    t['topic'] = i
    t = t.rename(columns = {i :'probability'})
    doctop.append(t)
doctop_df = pd.concat(doctop, axis=0)
doctop_df

,designation,probability,topic
P336801,Itinerary to Mazamua,0.984481,0
P334078,Measurements of Doorposts,0.982690,0
P335921,List of Textiles,0.982351,0
P335787,Textile Label,0.981630,0
P335786,Textile Label,0.979068,0
...,...,...,...
P336309,Report on Rituals Performed on Shebat 17-21,0.789558,9
P313842,Building Progress Report,0.783128,9
P335751,Building Progress Report,0.776834,9
P282261,Text Similar to No. 52,0.767442,9


# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

The LDAvis package, used in notebook 5.3, was originally written for statistical programming language R; pyLDAvis is a python wrapper. In R indexing starts with 1, rather than 0 (as is the case in Python). In order to prevent confusion, we will rename all topics according to the pyLDAvis convention.

In [17]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

,1,2,3,4,5,6,7,8,9,10
P224395,0.004001,0.240667,0.110501,0.004002,0.004001,0.004002,0.004002,0.620818,0.004001,0.004003
P224403,0.322458,0.615989,0.007693,0.007695,0.007693,0.007693,0.007695,0.007696,0.007694,0.007694
P224417,0.206175,0.002041,0.002042,0.075811,0.002041,0.002041,0.002041,0.703724,0.002042,0.002041
P224431,0.002174,0.002174,0.002174,0.263511,0.002174,0.002174,0.002174,0.719095,0.002174,0.002174
P224433,0.004763,0.004763,0.004763,0.183521,0.004763,0.004763,0.069543,0.713596,0.004763,0.004763
...,...,...,...,...,...,...,...,...,...,...
P452805,0.007144,0.545832,0.007146,0.007145,0.007147,0.397005,0.007146,0.007145,0.007144,0.007147
P452901,0.007695,0.007695,0.007695,0.007696,0.007697,0.007697,0.413542,0.443770,0.088817,0.007696
Q009252,0.036023,0.001451,0.001450,0.001450,0.493684,0.001451,0.321610,0.139980,0.001451,0.001451
X210106,0.002327,0.002327,0.002327,0.002327,0.002327,0.045755,0.791208,0.115421,0.033654,0.002327


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [18]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of tuples in the format (index, \[list\]). The index is the topic number, the list is again a list of tuples in the form `(lemma, probability)` in descending order of probability. This means that the lemmas are in a different order in each of the lists.

The command `pd.DataFrame(topic_term[i][1])` creates a DataFrame with two columns for the topic `i`. The first column (column 0) has the lemmas, the second column the probabilities. By using the command `set_index(0)` the lemmas become the index (rather than a column) and this index can be used to concatenate the DataFrames (one for each topic) with the `sort = True` option. 

In [19]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True, sort=True)
t_t_df_.head()

,0,1,2,3,4,5,6,7,8,9
0,,,,,,,,,,
ab2.ga[(meaning-and-reading-uncertain)]n,0.000026,0.000013,0.000007,0.000012,0.000093,0.000009,0.000006,0.000008,0.000006,0.000012
aban-bāšti[(a-stone)]n,0.000027,0.000013,0.000058,0.000012,0.000011,0.000009,0.000006,0.000008,0.000006,0.000012
aban-lamassi[(a-precious-stone)]n,0.000602,0.000013,0.000007,0.000012,0.000011,0.000009,0.000006,0.000008,0.000006,0.000012
aban-râmi['love'-stone]n,0.000027,0.000013,0.000058,0.000012,0.000011,0.000009,0.000006,0.000008,0.000006,0.000012
abati[(meaning-unknown)]n,0.000026,0.000013,0.000007,0.000096,0.000011,0.000009,0.000006,0.000008,0.000006,0.000012


Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [20]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

,ab2.ga[(meaning-and-reading-uncertain)]n,aban-bāšti[(a-stone)]n,aban-lamassi[(a-precious-stone)]n,aban-râmi['love'-stone]n,abati[(meaning-unknown)]n,abašmû[(a-stone)]n,abbušu[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,...,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭīpu[addition]n,ṭīru[impression]n,ṭūbtu[peace]n,ṭūbu[goodness]n,ṭūbātu[happiness]n,ṭūdu[way]n,ṭūru[opopanax]n
1,0.000026,0.000027,0.000602,0.000027,0.000026,0.000027,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000026,0.000027,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
2,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000628,...,0.000628,0.000013,0.000013,0.000013,0.000110,0.000013,0.027118,0.000013,0.000013,0.000110
3,0.000007,0.000058,0.000007,0.000058,0.000007,0.000058,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.001657,0.000007,0.000059,0.000007,0.000007,0.000346,0.000478,0.000007,0.000007
4,0.000012,0.000012,0.000012,0.000012,0.000096,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
5,0.000093,0.000011,0.000011,0.000011,0.000011,0.000011,0.000184,0.000011,0.000011,0.000011,...,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000964,0.000011,0.000011,0.000011
6,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000664,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000076,0.000009,0.000009,0.000009,0.000009
7,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.002075,0.000006,0.000006,...,0.000006,0.000322,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
8,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000323,0.000008,...,0.000008,0.000008,0.000242,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
9,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,...,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
10,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000185,0.000012


In [21]:
#just checking
t_t_df['ēkallu[palace]n']

1     0.003224
2     0.006878
3     0.000236
4     0.011354
5     0.003244
6     0.001480
7     0.010360
8     0.014353
9     0.003212
10    0.007336
Name: ēkallu[palace]n, dtype: float32

# Export Data

In [22]:
topic_model = {'dictionary': dictionary,
                  'corpus' : corpus,
                  'ldamodel' : ldamodel,
                  't_t_df' : t_t_df,
                  'd_t_df' : d_t_df,
               'df' : df,
              'ntopics' : ntopics,
              'texts' : texts}

In [23]:
with open('output/topic_model.p', 'wb') as w:
    pickle.dump(topic_model, w)